In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [25]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)  

In [3]:
# Visit https://redplanetscience.com/
url = 'https://redplanetscience.com/'
browser.visit(url)

time.sleep(1)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser") 

In [4]:
# Get the News Title
news_title = soup.find('div', class_='content_title').text

In [5]:
# Get the title intro
news_intro = soup.find('div', class_='article_teaser_body').text

In [6]:
# Visit https://spaceimages-mars.com/
url = 'https://spaceimages-mars.com/'
browser.visit(url)

time.sleep(1)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [7]:
# Find the featured image
featured_image = soup.find('img', class_='headerimage')

In [8]:
# Save the url to the image
featured_image_url = url + featured_image['src']

In [9]:
# Move on to the next website
url = 'https://galaxyfacts-mars.com/'

In [10]:
# Read in the tables from the site
tables = pd.read_html(url)

In [11]:
# Save the desired table
mars_facts = tables[0]

In [12]:
# Set the column names correctly
mars_facts.columns = mars_facts.iloc[0]

In [13]:
# Drops the row used as the columns
mars_facts.drop(mars_facts.index[0], inplace=True)

In [14]:
# Remove column name that isn't needed
mars_facts.rename(columns={ 'Mars - Earth Comparison' : '' }, inplace=True)

In [15]:
# Save the table to html without index so it looks nicer
mars_facts_html = mars_facts.to_html(index=False)

In [26]:
# Next website
url = 'https://marshemispheres.com/'

In [27]:
browser.visit(url)

time.sleep(1)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [18]:
# variables list for getting links and titles
product_list = []
url_list = []
product_title = []
product_url_list = []
hemisphere_image_urls = []

In [28]:
# get each hemisphere page available
product_items = soup.find_all('div', class_='description')

In [29]:
# loop to get each hemisphere page available
for product in product_items:
    title = product.find('h3').text
    product_title.append(title)
    product_url = product.find('a')['href']
    url_list.append(product_url)

product_url_list = [ url + p_url for p_url in url_list]

In [31]:
browser.links.find_by_partial_text(product_title[0]).click()
html = browser.html
soup = bs(html, "html.parser")

In [21]:
# Ensure the material is still there
try:
    for title in product_title:
        browser.links.find_by_partial_text(title).click()
        html = browser.html
        soup = bs(html, "html.parser")
        title = soup.find('h2', class_='title').text
        image = soup.find('div', id='wide-image').find_all('img')[1]['src']
        
        hemisphere_image_urls.append({ 'title' : title, 'img_url' : url + image})
        browser.links.find_by_partial_text('Back').click()
        
except:
    print("Scraping Complete")

In [22]:
# Store data in a dictionary
mars_data_dict = {
    "Featured_img": featured_image_url,
    "NewsTitle": news_title,
    "NewsIntro": news_intro,
    "Mars_Facts" : mars_facts_html,
    "Hemisphere_Imgs" : hemisphere_image_urls
}

In [48]:
# Close the browser after scraping
browser.quit()

In [24]:
mars_data_dict

{'Featured_img': 'https://spaceimages-mars.com/image/featured/mars2.jpg',
 'NewsTitle': 'Heat and Dust Help Launch Martian Water Into Space, Scientists Find',
 'NewsIntro': 'Scientists using an instrument aboard NASA’s Mars Atmosphere and Volatile EvolutioN, or MAVEN, spacecraft have discovered that water vapor near the surface of the Red Planet is lofted higher into the atmosphere than anyone expected was possible. ',
 'Mars_Facts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th></th>\n      <td>Diameter:</td>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th></th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th></th>\n      <td>Moons:</td>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th></th>\n 

In [ ]:
for img in mars_data_dict['Hemisphere_Imgs']:
    print(img['img_url'])